## Preambule

In [1]:
import numpy as np
from tqdm import tqdm
from importlib import reload
import xarray as xr
import pandas as pd
import scipy
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Sobol analysis
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Read stuff

In [2]:
path_data = "K:/ECEMF/T5.2/"

In [3]:
xr_dataread = xr.open_dataset(path_data + "xr_dataread.nc")
xr_sobol = xr.open_dataset(path_data + "xr_sobol.nc")
all_countries_iso = np.load(path_data + "all_countries.npy", allow_pickle=True)
all_regions_iso = np.load(path_data + "all_regions.npy", allow_pickle=True)

## Process stuff

In [4]:
dim_unc_globe = ['Temperature', 'Risk']
dim_unc_fairness = ['variable', 'Convergence_year']
dim_unc_choices = ['Timing', 'Scenario', 'NonCO2red', 'NegEmis']

vals = np.zeros(shape=(3, len(xr_sobol.Time), len(all_countries_iso)))
for dim_i, dim in enumerate([dim_unc_globe, dim_unc_fairness, dim_unc_choices]):
    vals[dim_i] = np.array(xr_sobol.sel(Factor = dim, Region=all_countries_iso).sum(dim='Factor').to_array())[0].T

In [5]:
mins = np.nanmin(vals, axis=2)
maxs = np.nanmin(vals, axis=2)
mins = (np.array(mins)*100).round(0).astype(int).astype(str)
maxs = (np.array(maxs)*100).round(0).astype(int).astype(str)

In [6]:
df_regions = pd.read_csv("X:/user/dekkerm/Data/AllCountries.csv")
continents = []
bubblecolors = []
texts = []
for cty_i, cty in enumerate(all_countries_iso):
    if cty == 'USA':
        bubblecolors.append('plum')
        texts.append(cty)
    elif cty == 'IND':
        bubblecolors.append('forestgreen')
        texts.append(cty)
    elif cty == 'BRA':
        bubblecolors.append('goldenrod')
        texts.append(cty)
    elif cty == 'CHN':
        bubblecolors.append('tomato')
        texts.append(cty)
    elif cty == 'ZAF':
        bubblecolors.append('steelblue')
        texts.append(cty)
    elif cty == 'MEX':
        bubblecolors.append('lightgreen')
        texts.append(cty)
    elif cty == 'TUR':
        bubblecolors.append('brown')
        texts.append(cty)
    elif cty == 'DEU':
        bubblecolors.append('dodgerblue')
        texts.append(cty)
    else:
        bubblecolors.append('silver')
        texts.append('')
continents = np.array(continents)
bubblecolors = np.array(bubblecolors)
texts = np.array(texts)

pops = np.array(xr_dataread.Population.sel(Region=all_countries_iso, Time=2020, Scenario='SSP2'))
pops[np.isnan(pops)] = 1
pops_n = pops/np.nanmax(pops)

## Plot

In [7]:
val_i = xr_sobol.sel(Region='USA', Factor='variable').Sobol_index

In [10]:
def draw_line(fig, name, x1, x2, y1, y2, color):
    fig.add_shape(type="line", xref='paper', yref='paper',
                  x0=x1,
                  y0=y1,
                  x1=x2,
                  y1=y2,
                  line=dict(color=color, width=3))

In [16]:
fig = make_subplots(rows=4, cols=3, specs=[[{"type": "scatterternary", "colspan": 2, "rowspan": 4}, {}, {}],
                                           [{}, {}, {}],
                                           [{}, {}, {}],
                                           [{}, {}, {}]],
                    horizontal_spacing = 0, vertical_spacing=0.05, shared_xaxes=True)

# Panel a
for i in np.arange(0., 1.01, 0.25):
    args = dict(mode='lines', showlegend=False, hoverinfo='skip',
                                                    marker={'size': 15,
                                                            'color': 'black',
                                                            'opacity': 1,#0.9-i*0.03,
                                                            'line' :dict(width=0., color='silver')},
                                                    line={'width': 0.33, #np.mean(s[i:i+2])*0.5,
                                                            'color': 'black',
                                                            'dash': 'dot'},
                                                    textfont=dict(size=5,
                                                            color='black'))
    fig.add_trace(go.Scatterternary(a=[i, i], b=[0, 1-i], c=[1-i, 0], **args), 1, 1)
    fig.add_trace(go.Scatterternary(b=[i, i], a=[0, 1-i], c=[1-i, 0], **args), 1, 1)
    fig.add_trace(go.Scatterternary(c=[i, i], b=[0, 1-i], a=[1-i, 0], **args), 1, 1)

for cty_i, cty in enumerate(all_countries_iso):
        fig.add_trace(go.Scatterternary(a=vals[0, :, cty_i],
                                        b=vals[1, :, cty_i],
                                        c=vals[2, :, cty_i],
                                                        mode='lines+markers+text',
                                                        showlegend=False,
                                                        text=[texts[cty_i]]+['']*(len(vals[0])-1),
                                                        marker={'size': 6,
                                                                'color': bubblecolors[cty_i],
                                                                'opacity': 0.9,#0.9-i*0.03,
                                                                'line' :dict(width=0, color='black')},
                                                        line={'width': 0.5, #np.mean(s[i:i+2])*0.5,
                                                                'color': 'silver'},
                                                        textfont=dict(size=(pops_n[cty_i])**0.4*25+1,
                                                                color='black')), 1, 1)
for cty in ['USA', 'CHN', 'BRA', 'ZAF', 'IND', 'MEX', 'DEU', 'TUR']:
        cty_i = np.where(all_countries_iso == cty)[0][0]
        fig.add_trace(go.Scatterternary(a=vals[0, :, cty_i],
                                        b=vals[1, :, cty_i],
                                        c=vals[2, :, cty_i],
                                                        mode='lines+markers+text',
                                                        showlegend=False,
                                                        text=[texts[cty_i]]+['']*(len(vals[0])-1),#["'., str(i-2000) for i in [2040, 2050, 2060, 2070, 2080, 2090]],
                                                        marker={'size': [(pops_n[cty_i])**0.4*60+4]+[15]*(len(vals[0])-1),
                                                                'color': bubblecolors[cty_i],
                                                                'opacity': 0.9,#0.9-i*0.03,
                                                                'line' :dict(width=0.5, color='white')},
                                                        line={'width': 3, #np.mean(s[i:i+2])*0.5,
                                                                'color':bubblecolors[cty_i]},
                                                        textfont=dict(size=(pops_n[cty_i])**0.4*25+4,
                                                                color='black')), 1, 1)

fig.update_layout({
'plot_bgcolor':'rgb(243, 243, 243)',
'ternary':
{'sum':1,
'bgcolor':'white',
    # "paper_bgcolor":'rgba(0,0,0,0)',
    # "plot_bgcolor":'rgba(0,0,0,0)',
'aaxis':{'title': '', 'min': 0., 'titlefont':{'size': 25},#Climate target<br>
        'linewidth':0, 'ticks':'outside',
        'tickmode':'array','tickvals':np.arange(0, 1.01, 0.25), 'ticktext':[str(int(i*100))+'%' for i in np.arange(0, 1.01, 0.25)], 'tickfont':{'size':15}},
'baxis':{'title': '', 'min': 0, 'titlefont':{'size': 25},# &nbsp; &nbsp; &nbsp;#Model &nbsp; &nbsp;
        'linewidth':2, 'ticks':'outside',
        'tickmode':'array','tickvals':np.arange(0, 1.01, 0.25), 'ticktext':[str(int(i*100))+'%' for i in np.arange(0, 1.01, 0.25)],'tickangle':60, 'tickfont':{'size':15}},
'caxis':{'title': '', 'min': 0, 'titlefont':{'size': 25},#Other<br>scenario assumptions
        'linewidth':2, 'ticks':'outside',
        'tickmode':'array','tickvals':np.arange(0, 1.01, 0.25), 'ticktext':[str(int(i*100))+'%' for i in np.arange(0, 1.01, 0.25)],'tickangle':-60, 'tickfont':{'size':15}}}})

# Panel b-d
for cty_i, cty in enumerate(['USA', 'CHN', 'IND']):
        for factor_i, factor in enumerate(['variable', 'Convergence_year', 'Temperature', 'Risk', 'NegEmis', 'NonCO2red', 'Timing', 'Scenario']):
                col = ['steelblue', 'dodgerblue', 'darkred', 'tomato', 'forestgreen', 'lightgreen', 'yellowgreen', 'greenyellow'][factor_i]
                val_i = xr_sobol.sel(Region=cty, Factor=factor).Sobol_index
                if factor_i == 0: fill='tozeroy'
                else: fill = 'tonexty'
                fig.add_trace(go.Scatter(x=xr_sobol.Time,
                                        y=val_i,
                                        mode='lines',
                                        marker={'color':col},
                                        fill=fill,
                                        name=['Allocation rule', 'Convergence year', 'Temperature target', 'Climate sensitivity', 'Negative emissions', 'Non-CO<sub>2</sub> reduction', 'Timing of action', 'Socio-economic scenario'][factor_i],
                                        showlegend=[True, False, False][cty_i],
                                        stackgroup='one'
                                        ), [1, 2, 3][cty_i], [3, 3, 3][cty_i])

fig.update_geos(visible=False,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                resolution=110,
                showcoastlines=False,)

fig.update_layout(
    legend=dict(
        yanchor="top",
        y=0.29,
        xanchor="left",
        x=0.08,
        font=dict(size=15, color="black"),
    ),
    #template='ggplot2',
    height=900,
    width=1600,
)
fig.update_xaxes(visible=False)

args = dict(xref="paper", yref="paper", showarrow=False)
annotations = [
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=24), y=0.98, x=0.0, text="<b>(a) Factors influencing<br>   fair emissions allocations</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=24), y=1.04, x=0.85, text="<b>(b) United States</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=24), y=0.78, x=0.77, text="<b>(c) China</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=24), y=0.49, x=0.77, text="<b>(d) India</b>"),
    dict(text="Normative", x=0.28, y=1.10, **args, font=dict(color='black', size=25)),
    dict(text="allocation method, parameters", x=0.25, y=1.07, **args, font=dict(color='darkgray', size=16)),
    dict(text="Climatological", x=-0.13, y=0.0, **args, font=dict(color='black', size=25)),
    dict(text="temperature targets,<br>climate sensitivity ", x=-0.125, y=-0.05, **args, font=dict(color='darkgray', size=16)),
    dict(text="Technological,<br>socio-economic", x=0.83, y=0.0, **args, font=dict(color='black', size=25)),
    dict(text="scenario, non-CO2,<br>negative emissions, policy", x=0.84, y=-0.05, **args, font=dict(color='darkgray', size=16))
]
for ann in annotations:
    fig.add_annotation(**ann)

#draw_line(fig, "", 0.65, 0.24, 1, 0.4, "black")


fig.update_layout(margin={"r":0,"t":75,"l":200,"b":50})
fig.update_layout(
       legend=dict(x=0.5, y=0.95),
    paper_bgcolor='white',  # Set the background color
    plot_bgcolor='white'
)
fig.write_image("X:/user/dekkerm/Projects/EffortSharing/Figures/ECEMF_paper/Figure_2.svg")